In [1]:
# Carica il file csv e coonverto colonna dei prezzi in int
import pandas as pd
import re
import itertools

df=pd.read_csv("cosenza_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df['prezzo_iniziale'] = df['prezzo_iniziale'].apply(lambda x: int(re.search(r'\d+', x).group()))

In [2]:
#Prendo tutte le combinazioni di coppie di ricerche sulla stessa città
df_list = df['TIMESTAMP'].unique()
coppie_df = list(itertools.combinations(df_list, 2))
len(coppie_df)


1

In [5]:
# Creo un Dataframe per vedere differenze di prezzo tra diverse ricerche
list_dict_df = []
for i in coppie_df:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data','prezzo_iniziale'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    # Elimino hotel con stesso nome 
    righe_droppate = len(merged_df)
    merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)
    righe_droppate -= len(merged_df)
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"]).abs()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe 1"
    else: prezzi_inferiori = "Dataframe 2"
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df ['num_differenze_prezzo'] = len(merged_df)
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len
    dict_df['tipo_ricerche'] = df1['user_agent_type'].unique()[0] + "/"+ df2['user_agent_type'].unique()[0]
    dict_df['os'] = df1['os'].unique()[0] + "/"+ df2['os'].unique()[0]
    dict_df['username'] = df1['username'].unique()[0] + "/"+ df2['username'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    #dict_df['righe_duplicate'] = righe_droppate
    list_dict_df.append(dict_df)

df_differenze = pd.DataFrame(list_dict_df)   
pd.set_option('display.max_rows', None)
df_differenze.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo_ricerche,os,username,prezzi_inferiori
0,03-06-2024---19:11 *** 03-06-2024---19:13,15,7.066667,7.0,0.194805,mobile/desktop,Android/Windows,pantilaura56/pantilaura56,Dataframe 1


In [ ]:
# Trova le ricerche con maggior numero di sconti 
df_differenze_top = df_differenze[df_differenze['%_prezzi_diversi']>0.5]
df_differenze_top['ricerche_piu_presenti'] = df_differenze_top['coppia_ricerche'].str.replace('*** ','')
orari = df_differenze_top['ricerche_piu_presenti'].str.split().explode().value_counts()
orari



In [ ]:
# Elimino una o più ricerche dal dataframe
new_df = df_differenze.drop(df_differenze[df_differenze['coppia_ricerche'].str.contains("20:42")].index) 
new_df.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

In [ ]:
#Visualizza dataframe mobile/mobile o desktop/desktop
df_tipo_uguale = df_differenze[(df_differenze['tipo_ricerche']=="mobile/mobile") | (df_differenze['tipo_ricerche']=="desktop/desktop")]
df_tipo_uguale.sort_values(by="num_differenze_prezzo",ascending= False)

In [5]:
# Visualizza il numero di ricerche mobile e desktop
mobile = 0
desktop = 0
for i in df_list:
    df1 = df[df["TIMESTAMP"]==i]
    stringa = df1['user_agent_type'].unique()[0]
    if "mob" in stringa:
        mobile +=1
    else: desktop+=1
    print(df1['user_agent_type'].unique())

print("Mobile:", mobile,"Desktop:",desktop)

['desktop']
['mobile']
Mobile: 1 Desktop: 1
